In [ ]:
docker ps --filter="name=sql"

In [ ]:
if (!$OriginalProjectPath) {$OriginalProjectPath = (Get-Item .).FullName}
$WorkingDirectory = $OriginalProjectPath + '\WorkingDirectory\LocalDockerExample'
"OriginalProjectPath: $OriginalProjectPath"
"WorkingDirectory: $WorkingDirectory"

In [ ]:
cd $env:TEMP\tf
$TerraformExePath = (Get-ChildItem)[0].FullName


In [ ]:
"TerraformExePath: $TerraformExePath"

In [ ]:
.\terraform version

In [ ]:
$command = 'docker inspect --format="{{.ID}}" sql'
Write-Output "Executing the command:"
Write-Output $command
$DockerId = Invoke-Expression $command
Write-Output 'Docker Id in question:' $DockerId

In [ ]:
$content = Get-Content -Path "$WorkingDirectory\LocalDockerExampleOriginal.tf.bak"
$content | Set-Content "$WorkingDirectory\LocalDockerExample.tf"

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory init"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory import docker_container.arbitrarylabel $DockerId"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory show"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory plan"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are Expected ******"
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory show -no-color > $WorkingDirectory\LocalDockerExample.tf"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
(Get-Content -Raw "$WorkingDirectory\LocalDockerExample.tf")

In [ ]:
$NewContent = (
    @'
terraform {
    required_providers {
        docker = {
            source  = "kreuzwerker/docker"
            version = "~> 2.17.0"
        }
    }
}
provider "docker" {
  #host = "unix:///var/run/docker.sock"
  host    = "npipe:////.//pipe//docker_engine"
}

'@).ReplaceLineEndings() + (Get-Content -Raw -Path "$WorkingDirectory\LocalDockerExample.tf").ReplaceLineEndings()
$NewContent = $NewContent.Replace('gateway           =',"# gateway           =")
$NewContent = $NewContent.Replace('id                =',"# id                =")
$NewContent = $NewContent.Replace('ip_address        =',"# ip_address        =")
$NewContent = $NewContent.Replace('ip_prefix_length  =',"# ip_prefix_length  =")
$NewContent = $NewContent.Replace('network_data      =',"/*network_data      =")
$NewContent = $NewContent.Replace('network_mode      =',"*/`n    network_mode      =")
$NewContent | Set-Content "$WorkingDirectory\LocalDockerExample.tf"

In [ ]:
(Get-Content -Raw "$WorkingDirectory\LocalDockerExample.tf")

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory init -upgrade -input=false"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are NOT Expected ******"
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory plan -input=false"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are NOT Expected ******"
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory plan -out=tfplan -input=false"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are NOT Expected ******"
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory apply -input=false tfplan"
Write-Output "Executing the command:"
Write-Output $command
Write-Output "****** Errors are NOT Expected ******"
Invoke-Expression $command

In [ ]:
# Full Reset
if (!$WorkingDirectory) {exit} # Guard Clause
#Remove-Item -LiteralPath "$WorkingDirectory\.terraform" #-Force -Recurse -whatif